

- Total duration to be divided into 2 sections if there is a change from one emotional state to another during the piece
- Each section will be divided into (number of notes), which will be calculated based on the randomly selected range of note durations
- The last note of every section will be atleast 3 seconds long
- The section duration is reduced by a randomly selected note duration within the range

Alternatively:
- If the affect has low arousal, the notes/chords can all be set to a standard duration (3-5s) 
- If affect has high arousal, the duration of each note/chord is randomly sampled from the range of note durations
- This process continues until the final note is less than 3 seconds long, in which case the penultimate note is extended to the end of the audio

In [1]:
import numpy as np
import random as rm
import pandas as pd
from collections import Counter
np.random.seed(42)

# read file
data = pd.read_csv('Liverpool_band_chord_sequence.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Liverpool_band_chord_sequence.csv'

In [15]:
n = 3
chords = data['chords'].values
ngrams = zip(*[chords[i:] for i in range(n)])
bigrams = [" ".join(ngram) for ngram in ngrams]

bigrams[:5]

['F Em7 A7', 'Em7 A7 Dm', 'A7 Dm Dm7', 'Dm Dm7 Bb', 'Dm7 Bb C7']

In [30]:
def predict_next_state(chord:str, data:list=bigrams, segment:int=1):
    """Predict next chord based on current state."""
    # create list of bigrams which stats with current chord
    bigrams_with_current_chord = [bigram for bigram in bigrams if bigram.split(' ')[0]==chord]
    # count appearance of each bigram
    count_appearance = dict(Counter(bigrams_with_current_chord))
    # convert apperance into probabilities
    for ngram in count_appearance.keys():
        count_appearance[ngram] = count_appearance[ngram]/len(bigrams_with_current_chord)
    
    # create list of possible options for the next chord
    options = [key.split(' ')[1] for key in count_appearance.keys()]
    # print(options)
    # create  list of probability distribution
    probabilities = list(count_appearance.values())
    # return random prediction
    return np.random.choice(options, p=probabilities)

In [31]:
predict_next_state('F')

'Fsus4'

In [32]:
def generate_sequence(chord:str=None, data:list=bigrams, length:int=30):
    """Generate sequence of defined length."""
    # create list to store future chords
    chords = []
    chords.append(chord)
    for n in range(length):
        # append next chord for the list
        chords.append(predict_next_state(chord, bigrams))
        # use last chord in sequence to predict next chord
        chord = chords[-1]
    return chords

In [33]:
generate_sequence('C')

['C',
 'Dm7',
 'G7',
 'Bb',
 'Dm',
 'Dm7',
 'Bb',
 'Dm',
 'Dm7',
 'Bb',
 'F',
 'Fsus4',
 'F',
 'F',
 'Em7',
 'A7',
 'Dm',
 'Dm7',
 'G7',
 'Bb',
 'Dm',
 'Gm6',
 'C7',
 'F',
 'C',
 'Bb',
 'Dm',
 'Gm6',
 'C7',
 'F',
 'F']

# Circle of fifths based approach

In [34]:
from collections import OrderedDict
major_keys = OrderedDict({
                'F':['Bb','F','C',"Dm"], # Each list contains keys that it can transition to (the first 3 are major and the last is minor)
                'C':['F','C','G',"Am"],
                'G':['C','G','D',"Em"],
                'D':['G','D','A',"Bm"],
                'A':['D','A','E',"F#m"],
                'E':['A','E','B',"C#m"],
                'B':['E','B','Gb',"G#m"],
                'Gb':['B','Gb','Db',"Ebm"],
                'Db':['Gb','Db','Ab',"Bbm"], 
                'Ab':['Db','Ab','Eb',"Fm"], 
                'Eb':['Ab','Eb','Bb',"Cm"], 
                'Bb':['Eb','Bb','F',"Gm"],
                'Am':["C","C","C","C"],
                'Em':["G","G","G","G"],
                'Bm':["D","D","D","D"],
                'F#m':["A","A","A","A"],
                'C#m':["Eb","Eb","Eb","Eb"],
                'G#m':["B","B","B","B"],
                'Ebm':["Gb","Gb","Gb","Gb"], 
                'Bbm':["Db","Db","Db","Db"], 
                'Fm':["Ab","Ab","Ab","Ab"], 
                'Cm':["Eb","Eb","Eb","Eb"],
                'Gm':["Bb","Bb","Bb","Bb"],
                'Dm':["F","F","F","F"]
            })
minor_keys = OrderedDict({
                'Am':['Dm','Am','Em',"C"],
                'Em':['Am','Em','Bm',"G"],
                'Bm':['Em','Bm','F#m',"D"],
                'F#m':['Bm','F#m','C#m',"A"],
                'C#m':['F#m','C#m','G#m',"Eb"],
                'G#m':['C#m','G#m','Ebm',"B"],
                'Ebm':['G#m','Ebm','Bbm',"Gb"], 
                'Bbm':['Ebm','Bbm','Fm',"Db"], 
                'Fm':['Bbm','Fm','Cm',"Ab"], 
                'Cm':['Fm','Cm','Gm',"Eb"],
                'Gm':['Cm','Gm','Dm',"Bb"],
                'Dm':['Gm','Dm','Am',"F"],
                'F':["Dm","Dm","Dm","Dm"], 
                'C':["Am","Am","Am","Am"],
                'G':["Em","Em","Em","Em"],
                'D':["Bm","Bm","Bm","Bm"],
                'A':["F#m","F#m","F#m","F#m"],
                'E':["C#m","C#m","C#m","C#m"],
                'B':["G#m","G#m","G#m","G#m"],
                'Gb':["Ebm","Ebm","Ebm","Ebm"],
                'Db':["Bbm","Bbm","Bbm","Bbm"], 
                'Ab':["Fm","Fm","Fm","Fm"], 
                'Eb':["Cm","Cm","Cm","Cm"], 
                'Bb':["Gm","Gm","Gm","Gm"]
                })
transition_probabilities=[0.3,0.3,0.3,0.1]

major_minor_transition = {
                            "F":"Dm",
                            "C":"Am",
                            "G":"Em",
                            "D":"Bm",
                            "A":"F#m",
                            "E":"C#m",
                            "B":"G#m",
                            "Gb":"Ebm",
                            "Db":"Bbm",
                            "Ab":"Fm",
                            "Eb":"Cm",
                            "Bb":"Gm"
                        }

In [32]:
list(OrderedDict(major_keys).keys())[:12]

['F', 'C', 'G', 'D', 'A', 'E', 'B', 'Gb', 'Db', 'Ab', 'Eb', 'Bb']

Algorithm:  
- Select a key randomly from the circle of fifths
- select a random number of notes from the key
- Add a list of these notes to a list
- return to the root note
- if required, use the major_minor_transition to change from major to minor keys
- repeat the process until the total duration of the piece is reached

In [41]:
def flatten(l):
    return [item for sublist in l for item in sublist]
def generate_sequence(starting_key, length):
    """Generate sequence of defined length."""
    # create list to store future chords
    chords = []
    current_key = starting_key
    # n_chords = rm.randint(2,10)
    # generate chords progressions for current_key

    chords.append(current_key)
    if current_key in list(major_keys.keys())[:12]:
        for n in range(length):
            # append next chord for the list
            choice = np.random.choice(major_keys[current_key], p=transition_probabilities)
            while (choice == current_key):
                choice = np.random.choice(major_keys[current_key], p=transition_probabilities)
            chords.append(choice)
            # use last chord in sequence to predict next chord
            current_key = chords[-1]
    elif current_key in list(minor_keys.keys())[:12]:
        for n in range(length):
            # append next chord for the list
            choice = np.random.choice(minor_keys[current_key], p=transition_probabilities)
            while (choice == current_key):
                choice = np.random.choice(minor_keys[current_key], p=transition_probabilities)
            chords.append(choice)
            # use last chord in sequence to predict next chord
            current_key = chords[-1]
        
    return chords
        

        



In [59]:
key = np.random.choice(list(OrderedDict(major_keys).keys())[:12])
generate_sequence(key, 10)

['F', 'C', 'F', 'Dm', 'F', 'C', 'F', 'C', 'F', 'Bb', 'Gm']

In [5]:
def flatten(l):
    return [item for sublist in l for item in sublist]
len(flatten([[1,2,3],[4,5,6],[7,8,9]]))

9

In [8]:
len(flatten({1:[1,2,3], 2:[4,5,6], 3:[7,8,9]}.values()))

9

In [1]:
import music21

In [4]:
# class using music21 to load a list of chords and play them as a midi object
class ChordPlayer:
    def __init__(self, chords):
        self.chords = chords
        self.stream = music21.stream.Stream()
        self.stream.append(music21.instrument.Piano())
    def play_chords(self):
        for chord in self.chords:
            self.stream.append(music21.chord.Chord(chord))
        self.stream.show('midi')


In [5]:
chords = ['C', 'F', 'Dm', 'F', 'C', 'F', 'C']
cp = ChordPlayer(chords)

In [6]:
cp.play_chords()

AccidentalException: m is not a supported accidental type